# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#import the cities _output.csv file from Part 1
cities_df = pd.read_csv('../output_data/cities_output.csv')
cities_df

,Unnamed: 0,country,city,lat,lon,date,cloudiness,humidity,max_temp,wind_speed
0,0,GL,qaanaaq,77.4840,-69.3632,1643577032,100,67,-7.91,6.64
1,1,RU,nikolskoye,59.7035,30.7861,1643577032,100,95,27.39,6.73
2,2,ID,bengkulu,-3.8004,102.2655,1643577032,100,89,75.97,3.27
3,3,US,menomonie,44.8755,-91.9193,1643577033,100,63,26.65,3.44
4,4,MV,hithadhoo,-0.6000,73.0833,1643576866,81,70,81.61,9.04
...,...,...,...,...,...,...,...,...,...,...
585,585,PG,lorengau,-2.0226,147.2712,1643577957,100,87,78.30,12.01
586,586,NG,gashua,12.8710,11.0482,1643578020,18,14,64.99,10.38
587,587,US,wahiawa,21.5028,-158.0236,1643578020,75,61,75.87,9.22
588,588,GD,saint-georges,12.0564,-61.7485,1643578021,20,74,82.08,14.97


In [3]:
#drop the duplicate index column created during the csv process in Part !
cities_df.drop('Unnamed: 0', inplace=True, axis=1)


In [4]:
display(cities_df)

,country,city,lat,lon,date,cloudiness,humidity,max_temp,wind_speed
0,GL,qaanaaq,77.4840,-69.3632,1643577032,100,67,-7.91,6.64
1,RU,nikolskoye,59.7035,30.7861,1643577032,100,95,27.39,6.73
2,ID,bengkulu,-3.8004,102.2655,1643577032,100,89,75.97,3.27
3,US,menomonie,44.8755,-91.9193,1643577033,100,63,26.65,3.44
4,MV,hithadhoo,-0.6000,73.0833,1643576866,81,70,81.61,9.04
...,...,...,...,...,...,...,...,...,...
585,PG,lorengau,-2.0226,147.2712,1643577957,100,87,78.30,12.01
586,NG,gashua,12.8710,11.0482,1643578020,18,14,64.99,10.38
587,US,wahiawa,21.5028,-158.0236,1643578020,75,61,75.87,9.22
588,GD,saint-georges,12.0564,-61.7485,1643578021,20,74,82.08,14.97


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
humidity = cities_df["humidity"].astype(float)
maxhumidity = cities_df["humidity"].astype(float).max()
locs_df = cities_df[["lat", "lon"]]
locs_df

,lat,lon
0,77.4840,-69.3632
1,59.7035,30.7861
2,-3.8004,102.2655
3,44.8755,-91.9193
4,-0.6000,73.0833
...,...,...
585,-2.0226,147.2712
586,12.8710,11.0482
587,21.5028,-158.0236
588,12.0564,-61.7485


In [6]:
maxhumidity

100.0

In [7]:
#https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html
#Best website for explaining how to use google maps
#Chose the center based on a value in dataframe--lat and lon
gmaps.configure(api_key = g_key)
figure = gmaps.figure(zoom_level=3,center=(48.8566,2.3522))
heat_layer = gmaps.heatmap_layer(locs_df, weights=humidity,dissipating=False,max_intensity=maxhumidity,point_radius=2)
figure.add_layer(heat_layer)
figure



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
cities_df

,country,city,lat,lon,date,cloudiness,humidity,max_temp,wind_speed
0,GL,qaanaaq,77.4840,-69.3632,1643577032,100,67,-7.91,6.64
1,RU,nikolskoye,59.7035,30.7861,1643577032,100,95,27.39,6.73
2,ID,bengkulu,-3.8004,102.2655,1643577032,100,89,75.97,3.27
3,US,menomonie,44.8755,-91.9193,1643577033,100,63,26.65,3.44
4,MV,hithadhoo,-0.6000,73.0833,1643576866,81,70,81.61,9.04
...,...,...,...,...,...,...,...,...,...
585,PG,lorengau,-2.0226,147.2712,1643577957,100,87,78.30,12.01
586,NG,gashua,12.8710,11.0482,1643578020,18,14,64.99,10.38
587,US,wahiawa,21.5028,-158.0236,1643578020,75,61,75.87,9.22
588,GD,saint-georges,12.0564,-61.7485,1643578021,20,74,82.08,14.97


In [9]:
perfect_df = cities_df.loc[
    (cities_df['max_temp'] > 60) &
    (cities_df['max_temp'] < 75) &
    (cities_df['wind_speed']<=20) &
    (cities_df['humidity']<=50)].reset_index(drop=True).dropna()

display(perfect_df)



,country,city,lat,lon,date,cloudiness,humidity,max_temp,wind_speed
0,US,san patricio,28.0170,-97.5169,1643577035,28,50,72.32,11.65
1,TD,am timan,11.0297,20.2827,1643577039,4,17,74.34,7.29
2,SS,tambura,5.6000,27.4667,1643577041,77,24,70.77,4.65
3,US,avera,33.1940,-82.5271,1643577047,0,43,60.39,16.22
4,AR,comodoro rivadavia,-45.8667,-67.5000,1643576340,40,27,64.26,13.80
5,BJ,nikki,9.9401,3.2108,1643577121,77,12,74.93,4.68
6,CF,paoua,7.2427,16.4406,1643577185,96,18,70.90,5.03
7,CM,ngaoundere,7.3167,13.5833,1643577193,87,20,66.29,6.85
8,SO,odweyne,9.4092,45.0640,1643577194,22,29,62.35,7.90
9,PT,ponta do sol,32.6667,-17.1000,1643577198,1,41,65.12,9.15


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = perfect_df

hotel_df['Hotel Name'] = ''

display(hotel_df.head())

,country,city,lat,lon,date,cloudiness,humidity,max_temp,wind_speed,Hotel Name
0,US,san patricio,28.0170,-97.5169,1643577035,28,50,72.32,11.65,
1,TD,am timan,11.0297,20.2827,1643577039,4,17,74.34,7.29,
2,SS,tambura,5.6000,27.4667,1643577041,77,24,70.77,4.65,
3,US,avera,33.1940,-82.5271,1643577047,0,43,60.39,16.22,
4,AR,comodoro rivadavia,-45.8667,-67.5000,1643576340,40,27,64.26,13.80,


In [11]:
radius = 5000
place = 'hotel'

for index, row in hotel_df.iterrows():
    lat = row['lat']
    lon = row['lon']
    
    query_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat}%2C{lon}&radius={radius}&type={place}&key={g_key}"

    result = requests.get(query_url).json()
    
    try:
        hotel_name = result['results'][1]['name']
    except:
        hotel_name = 'NA'
    
    hotel_df.at[index, 'Hotel Name'] = hotel_name


In [12]:
display(hotel_df)

,country,city,lat,lon,date,cloudiness,humidity,max_temp,wind_speed,Hotel Name
0,US,san patricio,28.0170,-97.5169,1643577035,28,50,72.32,11.65,Sinton High School
1,TD,am timan,11.0297,20.2827,1643577039,4,17,74.34,7.29,Yetibey Ngarsira
2,SS,tambura,5.6000,27.4667,1643577041,77,24,70.77,4.65,Ranzi Primary School
3,US,avera,33.1940,-82.5271,1643577047,0,43,60.39,16.22,"Hiram Bobo III, PC"
4,AR,comodoro rivadavia,-45.8667,-67.5000,1643576340,40,27,64.26,13.80,Austral Hotel
5,BJ,nikki,9.9401,3.2108,1643577121,77,12,74.93,4.68,Stade Bio Passo - Football Nikki
6,CF,paoua,7.2427,16.4406,1643577185,96,18,70.90,5.03,MSF FRANCE PAOUA Bureau
7,CM,ngaoundere,7.3167,13.5833,1643577193,87,20,66.29,6.85,Hotel Mentong Palace
8,SO,odweyne,9.4092,45.0640,1643577194,22,29,62.35,7.90,Xasaasi Media
9,PT,ponta do sol,32.6667,-17.1000,1643577198,1,41,65.12,9.15,Hotel do Campo


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [14]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
figure.add_layer(marker_layer)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))